In [5]:
%load_ext autoreload

In [6]:
import sys
sys.path.insert(0, '/home/divanik/CourseWork/Nystrom_TT_rounding_and_convolution')

In [7]:
import tt
import numpy as np
import time

In [8]:
%autoreload 2
from algorithms_package.src import hadamard_product, random_tensor_generation

In [9]:
def create_inverse_index_sums_tensorized(n, d, QTT_d, eps=1e-10):
    nats = tt.vector.to_list(tt.xfun(n, QTT_d))
    base = tt.vector.to_list(tt.ones(n, QTT_d * d))
    X = []
    for i in range(d):
        tmp = base.copy()
        for j in range(QTT_d):
            tmp[i * QTT_d + j] = nats[j].copy()
        X.append(tt.vector.from_list(tmp))
    c = tt.multifuncrs2(X, lambda x: 1 / (1 + np.sum(x, axis = 1)), eps=eps, verb=0)
    return c

In [48]:
t = create_inverse_index_sums_tensorized(2, 3, 20, 1e-12)

In [52]:
s = (t * t).round(0, 90)

In [12]:
u = tt.vector.to_list(t)

In [55]:
eps_rank_correspondence = {
    20 : 1e-4,
    23 : 1e-5,
    27 : 1e-6,
    30 : 1e-7,
    33 : 1e-8,
    36 : 1e-9,
    40 : 1e-10
}

In [56]:
def push(answer, name, precision, time):
    answer.append({
        'name' : name,
        'precision' : precision,
        'time' : time
    })
    return answer

In [58]:
eps_rank_correspondence.values()

dict_values([0.0001, 1e-05, 1e-06, 1e-07, 1e-08, 1e-09, 1e-10])

In [59]:
answer = []
for rank, eps in eps_rank_correspondence.items():
    time1 = time.time()
    a = (t * t).round(0, rank)
    time2 = time.time()
    b = tt.multifuncrs2([t, t], lambda x : x[:, 0] * x[:, 1], eps=eps, verb = 0)
    time3 = time.time()
    auxiliary_rank = int(1.5 * rank)
    c = hadamard_product.approximateTwoSidedHadamardProduct(u, u, [rank] * 59, [auxiliary_rank] * 59, rank * 10 + 1, True)[0]
    time4 = time.time()
    d = hadamard_product.approximateTwoSidedHadamardProduct(u, u, [rank] * 59, [rank] * 59, rank * 10 + 2, True)[0]
    time5 = time.time()
    e = hadamard_product.approximateHadamardProduct(u, u, [rank] * 59, rank * 10 + 3)
    time6 = time.time()
    c = tt.vector.from_list(c)
    d = tt.vector.from_list(d)
    e = tt.vector.from_list(e)
    time_round = time2 - time1
    time_multifuncrs = time3 - time2
    time_tsr_kappa_1_5 = time4 - time3
    time_tsr_kappa_1 = time5 - time4
    time_rto = time6 - time5
    precision_round = tt.vector.norm(s - a)
    precision_multifuncrs = tt.vector.norm(s - b)
    precision_tsr_kappa_1_5 = tt.vector.norm(s - c)
    precision_tsr_kappa_1 = tt.vector.norm(s - d)
    precision_rto = tt.vector.norm(s - e)
    answer = push(answer, 'round', time_round, precision_round)
    answer = push(answer, 'multifuncrs', time_multifuncrs, precision_multifuncrs)
    answer = push(answer, 'tsrh(kappa=1.5)', time_tsr_kappa_1_5, precision_tsr_kappa_1_5)
    answer = push(answer, 'tsrh(kappa=1)', time_tsr_kappa_1, precision_tsr_kappa_1)
    answer = push(answer, 'rtoh', time_rto, precision_rto)

In [60]:
import pandas as pd

In [61]:
df = pd.DataFrame(answer)

In [62]:
df

,name,precision,time
0,round,35.730229,3.112914e-06
1,multifuncrs,1.427439,2.136632e-03
2,tsrh(kappa=1.5),1.464997,8.284278e-04
3,tsrh(kappa=1),1.123807,3.271108e+03
4,rtoh,0.905817,8.008310e-05
5,round,35.387338,4.766424e-07
6,multifuncrs,1.539318,1.089000e-03
7,tsrh(kappa=1.5),1.610281,1.290354e-04
8,tsrh(kappa=1),1.277757,6.853811e+02
9,rtoh,1.003879,8.079366e-06


In [63]:
df = df.rename(columns={"precision" : "time", "time" : "precision"})

In [65]:
df.to_csv('/home/divanik/CourseWork/Nystrom_TT_rounding_and_convolution/data/experiment3/comparison.csv')

In [43]:
tt.vector.norm(s - a)

0.0014833775736754432

In [16]:
u = tt.vector.to_list(t)

In [44]:
kek0 = hadamard_product.approximateHadamardProduct(u, u, [20] * 59, 10)

In [45]:
kek1 = hadamard_product.approximateTwoSidedHadamardProduct(u, u, [20] * 59, [30] * 59, 10, True)[0]

In [46]:
tt.vector.from_list(kek1)

This is a 60-dimensional tensor 
r(0)=1, n(0)=2 
r(1)=20, n(1)=2 
r(2)=20, n(2)=2 
r(3)=20, n(3)=2 
r(4)=20, n(4)=2 
r(5)=20, n(5)=2 
r(6)=20, n(6)=2 
r(7)=20, n(7)=2 
r(8)=20, n(8)=2 
r(9)=20, n(9)=2 
r(10)=20, n(10)=2 
r(11)=20, n(11)=2 
r(12)=20, n(12)=2 
r(13)=20, n(13)=2 
r(14)=20, n(14)=2 
r(15)=20, n(15)=2 
r(16)=20, n(16)=2 
r(17)=20, n(17)=2 
r(18)=20, n(18)=2 
r(19)=20, n(19)=2 
r(20)=20, n(20)=2 
r(21)=20, n(21)=2 
r(22)=20, n(22)=2 
r(23)=20, n(23)=2 
r(24)=20, n(24)=2 
r(25)=20, n(25)=2 
r(26)=20, n(26)=2 
r(27)=20, n(27)=2 
r(28)=20, n(28)=2 
r(29)=20, n(29)=2 
r(30)=20, n(30)=2 
r(31)=20, n(31)=2 
r(32)=20, n(32)=2 
r(33)=20, n(33)=2 
r(34)=20, n(34)=2 
r(35)=20, n(35)=2 
r(36)=20, n(36)=2 
r(37)=20, n(37)=2 
r(38)=20, n(38)=2 
r(39)=20, n(39)=2 
r(40)=20, n(40)=2 
r(41)=20, n(41)=2 
r(42)=20, n(42)=2 
r(43)=20, n(43)=2 
r(44)=20, n(44)=2 
r(45)=20, n(45)=2 
r(46)=20, n(46)=2 
r(47)=20, n(47)=2 
r(48)=20, n(48)=2 
r(49)=20, n(49)=2 
r(50)=20, n(50)=2 
r(51)=20, n(51)=2 


In [47]:
tt.vector.norm(s - tt.vector.from_list(kek1))

0.0012014829486139923

In [40]:
tt.vector.norm(s - tt.vector.from_list(kek1))

3.948190550662471e-05

In [ ]:
kek0

In [55]:
p = (t * t).round(40)

: 

: 

([array([[[ 0.33178054,  0.1600103 , -0.38952485, -0.01461169,
            0.12609479, -0.17569242,  0.0540041 ,  0.03371427,
            0.01560122, -0.04507426,  0.10168334,  0.29181686,
           -0.24204171,  0.26132534,  0.07577827,  0.1104692 ,
           -0.20684377,  0.05562163,  0.34799387, -0.26457548,
           -0.47345728, -0.42502101,  0.09202403,  0.54571052,
            0.26350679,  0.38168693,  0.05043785,  0.29382291,
           -0.07346965,  0.14179319, -0.05494151, -0.11794095,
            0.02579787, -0.08701456,  0.3174004 ,  0.10978552,
            0.05566485, -0.06378624,  0.2700099 , -0.19360854],
          [ 0.24128329, -0.09150037, -0.33430285, -0.15357305,
           -0.23611919, -0.08524451, -0.09943244,  0.10063471,
            0.17722773, -0.05070754,  0.00305798,  0.12165217,
           -0.19410514,  0.24847527,  0.26985149,  0.07529077,
            0.72115588,  0.29101653,  0.00313083, -0.14137646,
           -0.12381132, -0.20223749,  0.36062492, -0.1

In [ ]:
import numpy as np
import tt
import experiments_stuff
from bilinear_package.src import primitives
from bilinear_package.src.hadamard_product import generalizedApproximateHadamardProduct
from bilinear_package.src.convolution import (
    approximateConvolution,
    preciseConvolution,
    countFourier,
    countInverseFourier,
    approximateCycleConvolution,
)
import time


from newton_experiment_primitives import create_exponential_grid, create_newtonial_potential_grid

from bilinear_package.src.convolution import preciseCycleConvolution
from newton_experiment_primitives import interpolateTTTensor, compressTTTensor
import pandas as pd

D = 40

df_time = pd.DataFrame()
df_precision = pd.DataFrame()

for d in range(3, 6):
    for grid_side in np.logspace(7, 10, 7, base=2, dtype=int):
        grid_size = 8 * (grid_side // 8)
        h = (2 * D) / grid_side
        tensor1 = create_exponential_grid(d, grid_side + 2, D + h / 2)
        tensor2 = create_newtonial_potential_grid(d, 2 * grid_side + 2, h * grid_side + h / 2)
        tensor1 = primitives.twoSidedPaddingTTTensor(tensor1, [(0, grid_side) for _ in range(d)])
        print(tt.vector.from_list(tensor1))
        print(tt.vector.from_list(tensor2))
        precise = preciseCycleConvolution(tensor1, tensor2)
        precise = primitives.twoSidedCuttingTTTensor(precise, [(0, grid_side + 1) for _ in range(d)])
        precise = tt.vector.from_list(precise)
        precise_norm = tt.vector.norm(precise)
        time1 = time.time()
        answer1 = approximateCycleConvolution(
            tensor1, tensor2, desired_ranks=np.ones(d - 1, dtype=int) * 30, seed=271 * d + grid_side
        )
        answer1 = primitives.twoSidedCuttingTTTensor(answer1, [(0, grid_side + 1) for _ in range(d)])
        time2 = time.time()
        answer2 = precise.round(0, 30)
        time3 = time.time()
        tensor1_ = tt.vector.from_list(countFourier(tensor1))
        tensor2_ = tt.vector.from_list(countFourier(tensor2))
        multifunc = tt.multifuncrs2([tensor1_, tensor2_], lambda x: x[:, 0] * x[:, 1], eps=1e-6, verb=0)
        answer3 = countInverseFourier(tt.vector.to_list(multifunc))
        answer3 = primitives.twoSidedCuttingTTTensor(answer1, [(0, grid_side + 1) for _ in range(d)])
        time4 = time.time()
        answer4 = approximateCycleConvolution(
            tensor1, tensor2, desired_ranks=np.ones(d - 1, dtype=int) * 30, seed=271 * d + grid_side
        )
        answer4 = primitives.twoSidedCuttingTTTensor(answer4, [(0, grid_side + 1) for _ in range(d)])
        time5 = time.time()
        df_time[f"{d}_{grid_side}"] = [time2 - time1, time3 - time2, time4 - time3]
        precision1 = tt.vector.norm(precise - tt.vector.from_list(answer1)) / precise_norm
        precision2 = tt.vector.norm(precise - answer2) / precise_norm
        precision3 = tt.vector.norm(precise - tt.vector.from_list(answer3)) / precise_norm
        df_precision[f"{d}_{grid_side}"] = [precision1, precision2, precision3]
        df_time.to_csv("experiments_results/experiment4/time")
        df_precision.to_csv("experiments_results/experiment4/precision")
        print(f"{d}_{grid_side}")
